In [1]:
from typing import Any

import numpy
import polars
from datasets import Dataset, load_dataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
dataset = load_dataset(
	"csv", data_dir="../data", data_files={"train": "train.csv", "test": "test.csv"}, keep_in_memory=True, num_proc=8
)

In [3]:
dataset_split = dataset["train"].train_test_split(test_size=0.1, keep_in_memory=True)
dataset["train"], dataset["validation"] = dataset_split["train"], dataset_split["test"]

In [4]:
index_columns: list = ["data_ID", "player_ID"]
label_columns: list = ["gender", "hold racket handed", "play years", "level"]
data_columns: list = list(set(dataset["train"].column_names).difference(set(index_columns + label_columns)))

In [5]:
train_dataset: polars.DataFrame = dataset["train"].to_polars()
train_data: numpy.ndarray = train_dataset[data_columns].to_numpy()
train_label: dict[str, numpy.ndarray] = {label: train_dataset[label].to_numpy() for label in label_columns}

In [6]:
validation_dataset: polars.DataFrame = dataset["validation"].to_polars()
validation_data: numpy.ndarray = validation_dataset[data_columns].to_numpy()
validation_label: dict[str, numpy.ndarray] = {label: validation_dataset[label].to_numpy() for label in label_columns}

In [7]:
hyperparameter_grid: dict[str, list[Any]] = {
	"n_estimators": [50, 100, 200, 300],
	"max_depth": [None, 10, 20, 30, 40],
	"min_samples_split": [2, 5, 10, 15],
	"min_samples_leaf": [1, 2, 4, 6],
	"max_features": ["sqrt", "log2"],
	"bootstrap": [True, False],
}

In [9]:
best_classifiers: dict = {}
for label in label_columns:
	print(f"Tune classifier for {label}")
	classifier = RandomForestClassifier(random_state=37)

	tuner = RandomizedSearchCV(
		classifier, hyperparameter_grid, n_iter=50, cv=5, scoring="roc_auc_ovr", n_jobs=-1, verbose=10
	)

	tuner.fit(train_data, train_label[label])

	best_classifiers[label] = tuner.best_estimator_

Tune classifier for gender
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for hold racket handed
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for play years
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Tune classifier for level
Fitting 5 folds for each of 1 candidates, totalling 5 fits


In [17]:
test_data: polars.DataFrame = dataset["test"].select_columns(data_columns).to_polars().to_numpy()

In [18]:
predictions: dict = {"data_ID": dataset["test"]["data_ID"]}
for label in label_columns:
	predictions[label] = best_classifiers[label].predict(test_data).tolist()

In [19]:
test_result = Dataset.from_dict(predictions)

In [20]:
test_result = test_result.map(
	lambda samples: {
		"play years_0": [1 if sample == 0 else 0 for sample in samples],
		"play years_1": [1 if sample == 1 else 0 for sample in samples],
		"play years_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["play years"],
	remove_columns=["play years"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

In [21]:
test_result = test_result.map(
	lambda samples: {
		"level_0": [1 if sample == 0 else 0 for sample in samples],
		"level_1": [1 if sample == 1 else 0 for sample in samples],
		"level_2": [1 if sample == 2 else 0 for sample in samples],
	},
	input_columns=["level"],
	remove_columns=["level"],
	batched=True,
	num_proc=8,
)

Map (num_proc=8):   0%|          | 0/7488 [00:00<?, ? examples/s]

In [22]:
test_result

Dataset({
    features: ['data_ID', 'gender', 'hold racket handed', 'play years_0', 'play years_1', 'play years_2', 'level_0', 'level_1', 'level_2'],
    num_rows: 7488
})

In [23]:
test_result.to_csv("../dist/result_rf.csv", num_proc=8)